# Paralelização e Distribuição do Processamento com MPI

## Geração do grafo

Ambos os grafos a serem utilizados serão os mesmos das abordagens anteriores.

## Código com MPI

In [2]:
%%writefile MPI1.cpp
#include <iostream>
#include <vector>
#include <fstream>
#include <algorithm>
#include <mpi.h>
#include <chrono>

using namespace std;
using namespace std::chrono;

vector<vector<int>> LerGrafo(const string &nomeArquivo, int &numVertices) {
    ifstream arquivo(nomeArquivo);
    int numArestas;
    arquivo >> numVertices >> numArestas;
    vector<vector<int>> grafo(numVertices, vector<int>(numVertices, 0));
    for (int i = 0; i < numArestas; ++i) {
        int u, v;
        arquivo >> u >> v;
        grafo[u - 1][v - 1] = 1;
        grafo[v - 1][u - 1] = 1;
    }
    arquivo.close();
    return grafo;
}

bool VerificaClique(const vector<vector<int>> &grafo, const vector<int> &clique) {
    for (int i = 0; i < clique.size(); i++) {
        for (int j = i + 1; j < clique.size(); j++) {
            if (grafo[clique[i]][clique[j]] == 0) { return false; }
        }
    }
    return true;
}

void EncontrarCliquesLocais(const vector<vector<int>>& grafo, const vector<int>& vertices_Iniciais, vector<vector<int>>& cliques_Locais) {
    vector<int> atualClique;

    auto backtrack = [&](int v, auto& backtrack_ref) -> void {
        // Salva o clique atual como uma solução válida
        if (!atualClique.empty()) { cliques_Locais.push_back(atualClique); }
        for (int i = v; i < grafo.size(); i++) {
            // Verifica se o vértice pode ser adicionado ao clique atual
            bool adiciona = true;
            for (int u : atualClique) {
                if (grafo[u][i] == 0) { // Não é clique
                    adiciona = false;
                    break;
                }
            }
            if (adiciona) {
                atualClique.push_back(i);
                backtrack_ref(i + 1, backtrack_ref);
                atualClique.pop_back();
            }
        }
    };
    // Pra cada vértice inicial, inicia a busca de cliques
    for (int vInicial : vertices_Iniciais) {
        atualClique.clear();
        atualClique.push_back(vInicial);
        backtrack(vInicial + 1, backtrack);
    }
}

int main(int argc, char **argv) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    high_resolution_clock::time_point inicio, fim;

    int numVertices = 0;
    vector<vector<int>> grafo;

    if (rank == 0) {
        grafo = LerGrafo("grafo1.txt", numVertices);
        
        inicio = high_resolution_clock::now();
        
        // Enviando numVertices para os processos
        for (int i = 1; i < size; i++) { MPI_Send(&numVertices, 1, MPI_INT, i, 0, MPI_COMM_WORLD); }

        // Enviado o grafo para os processos
        for (int i = 1; i < size; i++) {
            for (int j = 0; j < numVertices; j++) { MPI_Send(grafo[j].data(), numVertices, MPI_INT, i, 0, MPI_COMM_WORLD); }
        }
    } else {
        // Receber o numVertices do processo raiz
        MPI_Recv(&numVertices, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        grafo.resize(numVertices, vector<int>(numVertices));
        // Receber o grafo do processo raiz
        for (int j = 0; j < numVertices; j++) { MPI_Recv(grafo[j].data(), numVertices, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE); }
    }

    // Dividir
    vector<int> verticesIniciais;
    for (int i = rank; i < numVertices; i += size) { verticesIniciais.push_back(i); }

    // Cada processo executa a busca de cliques
    vector<vector<int>> Locais;
    EncontrarCliquesLocais(grafo, verticesIniciais, Locais);

    if (rank == 0) {
        vector<vector<int>> Cliques;
        Cliques.insert(Cliques.end(), Locais.begin(), Locais.end());

        for (int i = 1; i < size; i++) {
            int tam;
            MPI_Recv(&tam, 1, MPI_INT, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            vector<int> recebidos(tam);
            MPI_Recv(recebidos.data(), tam, MPI_INT, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            for (int k = 0; k < tam; k) {
                int tamClique = recebidos[k++];
                vector<int> clique;
                for (int j = 0; j < tamClique; j++) { clique.push_back(recebidos[k++]); }
                Cliques.push_back(clique);
            }
        }
        
        size_t maior = 0;
        for (const auto& clique : Cliques) {
            if (clique.size() > maior) { maior = clique.size(); }  // Maior tamanho de clique
        }
        
        vector<int> cliquesMax;
        for (const auto& clique : Cliques) { // Primeira clique de maior tamanho encontrada
            if (clique.size() == maior) { cliquesMax = clique; }
        }

        fim = high_resolution_clock::now();
        duration<double> duracao = duration_cast<duration<double>>(fim - inicio);

        cout << "Clique máxima arquivo saída: ";
        for (int m = 0; m < cliquesMax.size(); m++) { cout << cliquesMax[m]+1 << " "; }
        cout << endl;
        cout << "Tempo de execução: " << duracao.count() << " segundos" << endl;
    }
    else {
        vector<int> enviar;
        for (const auto& clique : Locais) {
            enviar.push_back(clique.size());
            enviar.insert(enviar.end(), clique.begin(), clique.end());
        }

        // Enviar o tamanho e os dados
        int tam = enviar.size();
        MPI_Send(&tam, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
        MPI_Send(enviar.data(), tam, MPI_INT, 0, 0, MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}

Writing MPI1.cpp


In [3]:
!mpic++ -o MPI MPI.cpp

In [4]:
!mpic++ -o MPI1 MPI1.cpp

## Criando arquivo de configuração

In [6]:
%%writefile submit1.slurm
#!/bin/bash
#SBATCH --job-name=mpi                          # Nome do job
#SBATCH --output=mpi_%j.txt                     # Nome do arquivo de saída
#SBATCH --ntasks=4                              # Número de tarefas 
#SBATCH --mem=2048                               # Memória total alocada por nó
#SBATCH --time=00:20:00                         # Tempo máximo de execução
#SBATCH --partition=espec                      # fila do cluster a ser utilizada

mpirun ./MPI1

Writing submit1.slurm


In [7]:
!sbatch submit.slurm

Submitted batch job 10023


In [8]:
!sbatch submit1.slurm

Submitted batch job 10024


## Realizando a verificação do grafo

In [9]:
with open('verificacao.txt', 'r') as file:
    lines = file.readlines()  # Lê todas as linhas
    ultima_linha = lines[-1]  # Pega a última linha
    _, numeros = ultima_linha.split("=", 1)
    lista = list(map(int, numeros.split()))
    lista.sort()
    print(lista)

with open('mpi_10023.txt', 'r') as file:
    lines = file.readlines()  # Lê todas as linhas
    print(lines)

with open('verificacao1.txt', 'r') as file:
    lines = file.readlines()  # Lê todas as linhas
    ultima_linha = lines[-1]  # Pega a última linha
    _, numeros = ultima_linha.split("=", 1)
    lista = list(map(int, numeros.split()))
    lista.sort()
    print(lista)

with open('mpi_10024.txt', 'r') as file:
    lines = file.readlines()  # Lê todas as linhas
    print(lines)

[1, 12, 14, 20, 30, 32, 33, 38, 43, 47, 49]
['Clique máxima arquivo saída: 3 8 19 23 28 31 32 33 40 43 50 \n', 'Tempo de execução: 0.0704077 segundos\n']
[10, 11, 12, 15, 30, 32, 35, 41, 47, 58, 60, 61, 68, 91, 100]
['Clique máxima arquivo saída: 10 11 12 15 30 32 35 41 47 58 60 61 68 91 100 \n', 'Tempo de execução: 9.90859 segundos\n']


## Tempo

Com a implementação MPI realizada, afirma-se, baseando-se na Tabela a seguir, que apesar de não ter sido mais rápido que a implementação em OpenMP, ainda foi mais rápido que a exaustiva.

| Implementação | numVertices |   Tempo     |
|---------------|-------------|-------------|
| exaustiva     |     50      | 0.193033 s  |
| exaustiva     |     100     | 51.3523 s   |
| openMP        |     50      | 0.0220482 s |
| openMP        |     100     | 3.59543 s   |
| MPI           |     50      | 0.0704077 s |
| MPI           |     100     | 9.90859 s   |

O speed up resultante é:

| Implementação | numVertices | speed up |
|---------------|-------------|----------|
| openMP        |     50      |  8.76    |
| openMP        |     100     | 14.29    |
| MPI           |     50      |  2.74    |
| MPI           |     100     |  5.18    |

Logo, o MPI também acelerou a execução, mas com menor eficiência comparado ao OpenMP.

O OpenMP pode ter sido mais rápido que o MPI porque a tarefa pode ser mais eficiente em sistemas de memória compartilhada, onde os threads podem acessar rapidamente os dados e se comunicar com menos latência. Já o MPI sofre mais com a sobrecarga de comunicação, especialmente se a aplicação não for distribuída de maneira ideal.